In [8]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from string import punctuation,digits
import re
from keras.utils import to_categorical
#from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#from pandas_ml import ConfusionMatrix
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Embedding
# from keras.utils.np_utils import probas_to_classes
from sklearn import preprocessing
from keras.models import Sequential
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib import rcParams
from random import shuffle
# figure size in inches
rcParams['figure.figsize'] = 11.7,8.27
#import word2vecReader as godin_embedding
#from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
#import skopt
#from skopt import gp_minimize, forest_minimize
#from skopt.space import Real, Categorical, Integer
#from skopt.utils import use_named_argsfrom keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam


In [9]:
def remove_punctuation(s):
    list_punctuation = list(punctuation)
    for i in list_punctuation:
        s = s.replace(i,' ')
    return s

In [10]:
def clean_sentence(sentence):
    sentence = sentence.lower()
    #remove multiple repeat non num-aplha char !!!!!!!!!-->!
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
#     print(sentence)
    #removes alpha char repeating more than twice aaaa->aa
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
#     print(sentence)
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    tokens = [remove_punctuation(w) for w in tokens]
#     print(tokens)

    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
#     for w in stop_words:
#         print(w)
#     print(tokens)
    # filter out short tokens
#     tokens = [word for word in tokens if len(word) > 1]
#     print(tokens)
    remove_digits = str.maketrans('', '', digits)
#     print(tokens)
    tokens = [w.translate(remove_digits) for w in tokens]
    tokens = [w.strip() for w in tokens]
    tokens = [w for w in tokens if w!=""]
#     print(tokens)
    tokens = ' '.join(tokens)
    return tokens

In [11]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [12]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [13]:
#loading GloVe embedding
def load_GloVe_embedding(file_name):
    embeddings_index = dict()
    f = open(file_name,encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

In [14]:
# create a weight matrix for words in training docs
def get_GloVe_embedding_matrix(embeddings_index,vocab_size,tokenizer):
    embedding_matrix = np.zeros((vocab_size, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [15]:
#Title and Content

def work(sentence,X_train,embeddings_index_glove):#,X_test):
    sentence = [clean_sentence(x) for x in sentence]
    lengths = [len(s.split()) for s in sentence]
    print('max len = ',max(lengths))
    max_length  = max(lengths)
    
    #Training data
    tokenizer = create_tokenizer(X_train)
    vocab_size = len(tokenizer.word_index) + 1
    trainX = encode_text(tokenizer, X_train, max_length)

    #Testing data
    #tokenizer = create_tokenizer(X_test)
    #vocab_size = len(tokenizer.word_index) + 1
    #testX = encode_text(tokenizer, X_test, max_length)
    

    get_embeddings = get_GloVe_embedding_matrix(embeddings_index_glove,vocab_size,tokenizer)
    
    return tokenizer,max_length,vocab_size,get_embeddings
    
    
    

In [16]:
embeddings_index_glove = load_GloVe_embedding('glove.6B.100d.txt')

Loaded 400000 word vectors.


In [17]:
#Loading Data
df = pd.read_csv('cleaned_data1.csv')

In [19]:
#Finding sentence and content
sentence_title= df['Title']
sentence_content  = df['Content']
label= df['Type']

In [20]:
#Defining training and testing
X_train_title = sentence_title
Y_train_title = label
X_train_content = sentence_content
Y_train_content = label


In [21]:
tokenizer_title,length_title,vocab_size_title,get_embeddings_title = work(sentence_title,X_train_title,embeddings_index_glove)
#similarly for content
tokenizer_content,length_content,vocab_size_content,get_embeddings_content = work(sentence_content,X_train_content,embeddings_index_glove)

max len =  64
max len =  6435


In [38]:
#Loading KG Embedding
print('Loading Relation data')
X_fake = np.load('SpotFake-master/KG_ours/final_fake_entRel.npy',encoding='latin1')
X_real = np.load('SpotFake-master/KG_ours/final_real_entRel.npy',encoding = 'latin1')

Loading Relation data


In [49]:
rel_embeddings=np.append(X_fake,X_real)

In [22]:
#Applying VGG Model to Images
from PIL import Image as PImage
from os import listdir
from keras.models import Model
from keras.applications.vgg19 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import optimizers
#function to find the image vector fro pre-trained VGG19 Model
#Datasets of fake and real images for each article
#file being the name of the folder of images.
def image_embeddings(file):
    imagesList = listdir(file)
    model= VGG19(weights='imagenet')
    #model_ = Sequential()
    #for layer in model.layers[:-1]:  #-1 to get rid of last layer
       # model_.add(layer)
    #model_.summary() 
    
    final_image = []
    for img in imagesList:
        img= file+'/'+img
        #img = PImage.open(file +'/'+ img)
        image = load_img(img, target_size=(224, 224))
# convert the image pixels to a numpy array
        image = img_to_array(image)
# reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
        image = preprocess_input(image)
        yhat = model.predict(image)
# predict the probability across all output classes
        final_image.append(yhat)
# convert the probabilities to class labels
       
    return final_image

In [23]:
#Training on existing images
fake_pic= image_embeddings('pics/fake_pics')
real_pic =image_embeddings('pics/real_pics')
final_pict = fake_pic+real_pic
df['Image'] = final_pict

C:\Users\Anubha\Anaconda3\lib\site-packages\PIL\Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
C:\Users\Anubha\Anaconda3\lib\site-packages\PIL\Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [64]:
# define the model
def define_model(length_title, vocab_size_title, length_content , vocab_size_content,rel_embeddings,get_embeddings_title,get_embeddings_content,final_pict):
    #=========================================================================
	# channel 1 : title
	inputs1 = Input(shape=(length_title,))
	embedding1 = Embedding(vocab_size_title,100)(inputs1)
    #Filter size:4 
	conv1 = Conv1D(filters=32, weights=[get_embeddings_title],kernel_size=4, activation='relu')(embedding1)
	pool1 = MaxPooling1D(pool_size=2)(conv1)
	flat1 = Flatten()(pool1)
    
    #----------------------------------------------------------
	# channel 2 :  CONTENT + KG
	inputs2 = Input(shape=(length_content,))
	embedding2 = Embedding(vocab_size_content, 100)(inputs2)
	conv2 = Conv1D(filters=32,weights=[get_embeddings_content], kernel_size=4, activation='relu')(embedding2)
	#drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
    
	#should this be embedding content ka length?

	inputs1k = Input(shape=(length_content,))
	embedding1k = Embedding(vocab_size_content, 100)(inputs1k)
	conv1k = Conv1D(filters=32, kernel_size=3, weights=[rel_embeddings],activation='relu')(embedding1k)
	#drop1 = Dropout(0.5)(conv1)
	pool1k = MaxPooling1D(pool_size=2)(onv1k)
	flat1k = Flatten()(pool1k)
	# channel 2
	inputs2k = Input(shape=(length_content,))
	embedding2 = Embedding(vocab_size_content, 100)(inputs2)
	conv2k = Conv1D(filters=32, weights=[rel_embeddings],kernel_size=4, activation='relu')(embedding2k)
	#drop2 = Dropout(0.5)(conv2)
	pool2k = MaxPooling1D(pool_size=2)(conv2k)
	flat2k = Flatten()(pool2k)
	# channel 3
	inputs3k = Input(shape=(length_content,))
	embedding3k = Embedding(vocab_size_content, 100)(inputs3k)
	conv3k = Conv1D(filters=32, kernel_size=5, weights=[rel_embeddings],activation='relu')(embedding3k)
	#drop3 = Dropout(0.5)(conv3)
	pool3k = MaxPooling1D(pool_size=2)(conv3k)
	flat3k = Flatten()(pool3k)
	# merge
	mergedke = concatenate([flat1k, flat2k, flat3k])
    
    merged_content = concatenate([flat2,mergedke])
    #-------------------------------------------------------------
   
    #IMAGES
    inputs4 = Input(shape=len(image_embedding[0]),)#max length of  of image embedding
    
    image_embedding =final_pict

   #======================================================= 
	# merge
	merged = concatenate([flat1, merged_content ,image_embedding])
    #merged2= conactenate(image_embeddings)
	# interpretation
    
	dense1 = Dense(10, activation='relu')(merged)
    #dense2 = Dense(10,activation='relu')(merged2)
	outputs = Dense(1, activation='softmax')(dense1)
    #what should be put in input for  image embedding?
	model = Model(inputs=[inputs1, inputs2, inputs4], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='final.png')
	return model

IndentationError: unindent does not match any outer indentation level (<ipython-input-64-7954f2c1a20e>, line 46)